<a href="https://colab.research.google.com/github/Best-Island-LCH/AI_NLP_BuffaloWing/blob/feat%233_create_model_draft/%EB%B2%84%ED%8C%94%EB%A1%9C%EC%9C%99%EC%9D%80%EC%97%AD%EC%8B%9C%EC%82%AC%EC%84%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RLHF 학습 모델(초안)

태클환경 반박시 당신의 말이 맞습니다

참고문헌
https://colab.research.google.com/drive/106DU9bKWEDg3cjmNI2tYqZNHs_8BKKOI?usp=sharing#scrollTo=yzu5OH2mWgOs

https://www.notion.so/InstructGPT-2dee51228f4b80c9905acdcf2a59ed76#2dee51228f4b81f09262c6f8222eb4db

In [4]:
# 패키지 설치, 최초 한해서 설치하고, 이후에 주석 처리해주세요

# !pip install trl transformers datasets peft accelerate
# !pip install -q trl transformers accelerate git+https://github.com/huggingface/peft.git bitsandbytes
# !pip install -q -U bitsandbytes transformers peft accelerate trl
# !pip install -U flash-attn --no-build-isolation!pip uninstall -y trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [1]:
# 1. 구글 드라이브 마운트
from google.colab import drive
import os
import torch

drive.mount('/content/drive/')

# 2. 데이터 및 프로젝트 경로 설정
# 스크린샷 구조: 내 드라이브 > Buffalo_Wing 폴더 바로 안에 파일이 있음
BASE_DIR = "/content/drive/MyDrive/Buffalo_Wing"

# 3. 데이터 파일 경로 설정
# os.path.join을 사용하여 폴더 경로(BASE_DIR)와 파일명을 합쳐 절대 경로를 만듭니다.
DATA_PATHS = {
    "sft_data": os.path.join(BASE_DIR, "SFTdata.json"),
    "sft_label": os.path.join(BASE_DIR, "SFTlabel.json"),
    "rm_data": os.path.join(BASE_DIR, "RMdata.json"),
    "rm_label": os.path.join(BASE_DIR, "RMlabel.json"),
    "ppo_data": os.path.join(BASE_DIR, "PPOdata.json"),
}

# # 경로가 올바른지 확인 (True가 나와야 정상)
# print(f"SFT 라벨 파일 존재 여부: {os.path.exists(DATA_PATHS['sft_label'])}")
# print(f"설정된 경로: {DATA_PATHS['sft_label']}")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
# SFT파파파 파인튜닝

import json
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

# ==========================================
# 1. 모델 및 저장 경로 설정
# ==========================================

# [변경] 요청하신 모델 ID로 변경
MODEL_ID = "kakaocorp/kanana-1.5-8b-instruct-2505"

# 결과 저장 경로
OUTPUT_DIR = os.path.join(BASE_DIR, "kanana-1.5-8b-sft-checkpoint")

# ==========================================
# 2. 데이터 로드 및 전처리
# ==========================================
# (이전과 동일)

def load_and_process_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")

    with open(file_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    processed_data = []
    for item in raw_data['data_info']:
        try:
            question = item['question']
            answer = item['answer']['contents']
            text = f"### 질문: {question}\n\n### 답변: {answer}"
            processed_data.append({"text": text})
        except KeyError:
            continue

    return Dataset.from_list(processed_data)

print(f"데이터 로드 중: {DATA_PATHS['sft_label']}")
dataset = load_and_process_data(DATA_PATHS["sft_label"])
print(f"학습 데이터 개수: {len(dataset)}")

# ==========================================
# 3. 모델 및 토크나이저 로드, formatting_func생성
# ==========================================

# [변경] A100은 bfloat16이 지원되므로 compute_dtype을 bfloat16으로 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",# 선형 레이어를 nf4로 대체하여 4비트 양자화 설정?
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산방법을 bf16으로 설정?
)

# 문서를 뽑아서 텍스트로
def formatting_func(example):
    return example["text"]

# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"
tokenizer.model_max_length = 1024


# [변경] attn_implementation="flash_attention_2" 추가 (A100 필수 옵션)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2"
)

# ==========================================
# 4. LoRA 설정 (Rank 64 유지)
# ==========================================

peft_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# ==========================================
# 5. 학습 설정 (Training Arguments)
# ==========================================

# SFTConfig에는 '학습 하이퍼파라미터'만 남깁니다.
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    bf16=True,       # A100용
    fp16=False,
    logging_steps=10,
    save_strategy="epoch",
)


# ==========================================
# 6. 학습 시작
# ==========================================

# SFTTrainer에 '데이터 관련 설정'을 넣어줍니다.
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    peft_config=peft_config,
    processing_class=tokenizer,
    formatting_func=formatting_func,

    # max_seq_length=1024,
    # dataset_text_field="text",
    # packing=False,
)

print(f"모델({MODEL_ID}) SFT 학습을 시작합니다 (A100 Mode)...")
trainer.train()

trainer.save_model(OUTPUT_DIR)
print(f"학습 완료! 모델 저장 경로: {OUTPUT_DIR}")

데이터 로드 중: /content/drive/MyDrive/Buffalo_Wing/SFTlabel.json
학습 데이터 개수: 10580


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Applying formatting function to train dataset:   0%|          | 0/10580 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/10580 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10580 [00:00<?, ? examples/s]

...

모델(kakaocorp/kanana-1.5-8b-instruct-2505) SFT 학습을 시작합니다 (A100 Mode)...


Step,Training Loss
10,1.169000
20,1.043400
30,1.047300
40,1.024000
50,1.042800
60,1.104100
70,1.018800
80,1.035200
90,1.014300
100,1.045900


Step,Training Loss
10,1.169000
20,1.043400
30,1.047300
40,1.024000
50,1.042800
60,1.104100
70,1.018800
80,1.035200
90,1.014300
100,1.045900


wandb: WARNING URL not available in offline run


학습 완료! 모델 저장 경로: /content/drive/MyDrive/Buffalo_Wing/kanana-1.5-8b-sft-checkpoint


In [ ]:
#RM 만들기
# 마법의 주문 sudo rm -rf /

import os
import json
import torch
import numpy as np # [추가] 계산용
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from trl import RewardTrainer, RewardConfig
from peft import LoraConfig

# ==========================================
# 1. 설정 및 경로
# ==========================================
MODEL_ID = "kakaocorp/kanana-1.5-8b-instruct-2505"
OUTPUT_DIR = os.path.join(BASE_DIR, "kanana-1.5-8b-rm-checkpoint")

# ==========================================
# 2. 데이터 로드 및 전처리
# ==========================================
def load_rm_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    processed_data = []
    for item in raw_data['data_info']:
        try:
            question = item['question']
            answers = []
            for key, value in item.items():
                if key.startswith('answer') and isinstance(value, dict):
                    ranking = value.get('ranking')
                    content = value.get('contents')
                    if ranking is not None and content:
                        answers.append({'ranking': ranking, 'text': content})

            answers.sort(key=lambda x: x['ranking'])

            if len(answers) >= 2:
                processed_data.append({
                    "chosen": f"### 질문: {question}\n\n### 답변: {answers[0]['text']}",
                    "rejected": f"### 질문: {question}\n\n### 답변: {answers[-1]['text']}"
                })
        except KeyError:
            continue
    return Dataset.from_list(processed_data)

try:
    full_dataset = load_rm_data(DATA_PATHS["rm_label"])
    print(f"전체 데이터 개수: {len(full_dataset)}")

    # [추가] 학습용(90%)과 평가용(10%)으로 데이터 쪼개기
    # 평가용 데이터가 있어야 Accuracy를 잴 수 있습니다.
    dataset_split = full_dataset.train_test_split(ㅛ=0.1)
    train_dataset = dataset_split['train']
    eval_dataset = dataset_split['test']

    print(f"학습 데이터: {len(train_dataset)}개 / 평가 데이터: {len(eval_dataset)}개")

except Exception as e:
    print(f"데이터 로드 실패: {e}")

# ==========================================
# 3. 모델 및 토크나이저
# ==========================================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "right"

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=1,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2"
)
model.config.pad_token_id = tokenizer.pad_token_id

# ==========================================
# 4. LoRA 설정
# ==========================================
peft_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# ==========================================
# 5. [핵심] Accuracy 계산 함수 추가
# ==========================================
def compute_metrics(eval_pred):
    # 모델이 예측한 점수들을 가져옵니다.
    # predictions[0] = chosen 점수들
    # predictions[1] = rejected 점수들
    rewards_chosen = eval_pred.predictions[0]
    rewards_rejected = eval_pred.predictions[1]

    # chosen 점수가 rejected 점수보다 크면 정답(1), 아니면 오답(0)
    accuracy = np.mean(rewards_chosen > rewards_rejected)

    return {"accuracy": accuracy}

# ==========================================
# 6. 학습 설정 (Evaluation 추가)
# ==========================================
training_args = RewardConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    bf16=True,
    logging_steps=10,
    save_strategy="epoch",
    max_length=1024,
    center_rewards_coefficient=0.01,

    # [추가] 평가(Evaluation) 설정
    eval_strategy="steps", # 스텝마다 평가 진행
    eval_steps=500,         # 500 스텝마다 Accuracy 확인 (데이터 양에 따라 조절하세요)
    per_device_eval_batch_size=16,
)

# ==========================================
# 7. 학습 시작
# ==========================================
trainer = RewardTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, # [추가] 평가 데이터 전달
    args=training_args,
    peft_config=peft_config,
    processing_class=tokenizer,
    compute_metrics=compute_metrics, # [추가] 채점 함수 전달
)

print("RM 학습(with Accuracy Check)을 시작합니다...")
trainer.train()

trainer.save_model(OUTPUT_DIR)
print(f"RM 학습 완료! 저장 경로: {OUTPUT_DIR}")

전체 데이터 개수: 26408
학습 데이터: 23767개 / 평가 데이터: 2641개


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at kakaocorp/kanana-1.5-8b-instruct-2505 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Adding EOS to train dataset:   0%|          | 0/23767 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/23767 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

Filtering eval >1024 tokens:   0%|          | 0/2641 [00:00<?, ? examples/s]

RM 학습(with Accuracy Check)을 시작합니다...


Step,Training Loss,Validation Loss,Accuracy
500,0.529900,0.556350,1.000000
1000,0.535400,0.545846,1.000000


In [ ]:
import torch
import json
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from peft import PeftModel
from datasets import Dataset

# ==========================================
# 1. 경로 및 설정
# ==========================================

# [중요] 1단계에서 합체(Merge)한 SFT 모델 경로
SFT_MODEL_PATH = os.path.join(BASE_DIR, "kanana-1.5-8b-sft-merged")

# [중요] 2단계에서 학습한 RM(보상 모델) 경로
RM_ADAPTER_PATH = os.path.join(BASE_DIR, "kanana-1.5-8b-rm-checkpoint")

# 결과 저장 경로
OUTPUT_DIR = os.path.join(BASE_DIR, "kanana-1.5-8b-ppo-checkpoint")

# PPO 데이터 경로
PPO_DATA_PATH = DATA_PATHS["ppo_data"]

# ==========================================
# 2. 데이터 로드 및 전처리
# ==========================================

def build_dataset(data_path, tokenizer):
    with open(data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    questions = []
    for item in data['data_info']:
        try:
            q = item['question']
            # SFT때와 같은 프롬프트 형식을 씌워줍니다. (답변 부분은 비워둠)
            prompt = f"### 질문: {q}\n\n### 답변:"
            questions.append(prompt)
        except KeyError:
            continue

    # 데이터셋 생성
    ds = Dataset.from_dict({"query": questions})

    # 토크나이징 함수
    def tokenize(sample):
        # PPO는 질문(Query)을 모델에 넣어야 하므로 미리 토크나이징 합니다.
        sample["input_ids"] = tokenizer.encode(sample["query"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

# ==========================================
# 3. 모델 로드 (메모리 절약 모드)
# ==========================================

# 3-1. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(SFT_MODEL_PATH)
# PPO 생성 시 왼쪽 패딩이 필요할 수 있으나, trl 최신버전은 자동 처리해주기도 함.
# 안전을 위해 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3-2. PPO용 정책 모델 (Actor) 로드
# AutoModelForCausalLMWithValueHead는 모델 머리에 '가치 판단용 레이어'를 하나 더 붙인 것입니다.
print("SFT(Actor) 모델 로드 중...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLMWithValueHead.from_pretrained(
    SFT_MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 3-3. 보상 모델 (Reward Model) 로드
# RM은 학습하지 않고 '채점'만 하므로 gradient 계산을 끕니다.
print("RM(Critic) 모델 로드 중...")
rm_model = AutoModelForSequenceClassification.from_pretrained(
    SFT_MODEL_PATH, # 베이스는 SFT 모델 (구조 동일)
    num_labels=1,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
# 학습해둔 RM LoRA 어댑터 장착
rm_model = PeftModel.from_pretrained(rm_model, RM_ADAPTER_PATH)
rm_model.eval() # 평가 모드

# ==========================================
# 4. PPO 설정
# ==========================================

config = PPOConfig(
    model_name=SFT_MODEL_PATH,
    learning_rate=1.41e-5,
    batch_size=4,             # 메모리 부족 시 1 또는 2로 줄이세요 ->넉넉하면 8
    mini_batch_size=1,        # 업데이트 시 사용할 미니배치
    gradient_accumulation_steps=1,
    optimize_cuda_cache=True,
    early_stopping=False,
    target_kl=0.1,            # [중요] 모델이 너무 망가지지 않게 제어하는 값 (KL Divergence)
    ppo_epochs=1,             # 같은 배치를 몇 번 반복 학습할지
    seed=42,
)

# 데이터 준비
dataset = build_dataset(PPO_DATA_PATH, tokenizer)

# 콜레이터(배치 묶는 함수)
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# 트레이너 초기화
ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=None, # None이면 model을 복사해서 참조 모델로 씁니다 (메모리 주의)
    tokenizer=tokenizer,
    dataset=dataset,
    data_collator=collator,
)

# ==========================================
# 5. PPO 학습 루프 (수동 제어)
# ==========================================

# 생성 파라미터 (답변을 얼마나 길고 창의적으로 만들지)
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 256, # 답변 길이 제한
}

print('\033[38;5;208m' + 'PPO 학습을 힘! 찬! 함성과! 함께! 시자아아아악! 하겠습니다아아아아아!' +'\033[0m')

# 전체 데이터 순회
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    # 1. 답변 생성 (Rollout)
    response_tensors = ppo_trainer.generate(
        query_tensors,
        return_prompt=False,
        **generation_kwargs
    )

    # 2. 답변 텍스트로 변환 (RM 채점용)
    batch["response"] = tokenizer.batch_decode(response_tensors)
    batch["query"] = tokenizer.batch_decode(query_tensors)

    # RM에 넣기 위해 질문+답변 합치기
    texts_for_rewards = [q + r for q, r in zip(batch["query"], batch["response"])]

    # 3. 보상(Reward) 계산
    # RM 모델에 넣어서 점수를 받습니다.
    rewards = []

    # RM 추론은 메모리 절약을 위해 torch.no_grad()
    with torch.no_grad():
        inputs = tokenizer(texts_for_rewards, return_tensors="pt", padding=True, truncation=True).to("cuda")
        output = rm_model(**inputs)
        # 점수(logits)만 추출
        score = output.logits.squeeze(-1)

        for s in score:
            rewards.append(s) # 텐서 형태로 리스트에 저장

    # 4. PPO 업데이트 (핵심!)
    # 질문, 답변, 점수를 주면 알아서 모델을 업데이트합니다.
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

    # 로그 출력 (선택)
    if epoch % 10 == 0:
        print(f"Step {epoch}: Mean Reward = {torch.stack(rewards).mean().item():.4f}")

# ==========================================
# 6. 결과 저장
# ==========================================
print("학습 완료! 모델 저장 중...")
ppo_trainer.save_pretrained(OUTPUT_DIR)
print(f"저장 완료: {OUTPUT_DIR}")